E1
T1.1
ProductName:    	macOS### E1: Clock Granularity Tests

# T1.1: macOS Version and Timing Granularity
## System Information
- **macOS 15.0.1** (BuildVersion: 24A348)
  - Kernel Version: 24.0.0
  - Architecture: arm64
  - Clock Granularity:
    - `time.time`: 7.152557373046875e-07
    - `timeit.default_timer`: 8.297502063214779e-08
    - `time.time_ns`: 768.0

- **macOS 14.6.1** (BuildVersion: 23G93)
  - Kernel Version: 23.6.0
  - Architecture: arm64
  - Clock Granularity:
    - `time.time`: 7.152557373046875e-07
    - `timeit.default_timer`: 8.288770914077759e-08
    - `time.time_ns`: 768.0

### Code Used: `clock_granularity.py`
**Path:** `A1/E1/T1_1/clock_granularity.py`

---

### T1.2: JuliaSet Benchmarking

#### Command Run:
```python
python3 -m timeit -n 5 -r 1 -s "import JuliaSet" "JuliaSet.calc_pure_python(desired_width=1000, max_iterations=300)"
```

#### Results on 4.05 GHz Core:
- **Function: `calculate_z_serial_purepython`**
  - Average Execution Time: 2.619 seconds
  - Average Standard Deviation: 0.0103 seconds

- **Function: `calc_pure_python`**
  - Average Execution Time: 2.811 seconds
  - Average Standard Deviation: 0.0215 seconds

### Observations:
- The standard deviations of 10.3 ms and 21.5 ms are orders of magnitude larger than the clock cycle time (~0.25 ns at 4.05 GHz).
- Potential causes of variance:
  - Thread scheduling and kernel interference.
  - Variations in CPU core utilization (e.g., performance cores vs efficiency cores).

---

### T1.3: Profiling Results

#### cProfile Command:
```bash
python3 -m cProfile -s cumulative JuliaSet.py
```

**Observations:**
- `cProfile` introduces significant overhead compared to custom decorators but provides more detailed profiling data.

#### Line Profiler Command:
```bash
python3 -m kernprof -l JuliaSet.py
```

#### Overhead Measurements:
- **Without Profiler:** Average time: 2.325 seconds.
- **With `cProfile`:** Average time: 5.088 seconds (+2.763 seconds, ~219% increase).
- **With Line Profiler:** Average time: 26.15 seconds (+23.825 seconds, ~11247% increase).

---

### T1.4: Memory Profiling Results

#### Commands Used:
- **Memory Profiler:**
  ```bash
  python3 -m memory_profiler JuliaSet.py
  ```
- **mprof Plotting:**
  ```bash
  python3 -m mprof run JuliaSet.py
  ```

#### Observations:
- **Memory Profiler Run:**
  - Single run: 1578.77 seconds (1576.445 seconds longer than without profiling).
  - Overhead: ~679x.
- **mprof Command:**
  - Average time after 5 runs: 2.337 seconds.
  - Overhead: ~0.5%, negligible in comparison.

---

### E2: Profiling Diffusion Equation Code

#### cProfile Visualization:
**Tool Used:** SnakeViz

#### Line Profiler Results:
Results presented as line-by-line execution times.

#### Memory Profiler:
- **Command:**
  ```bash
  python3 -m memory_profiler diffusion.py
  ```
- **mprof Plotting:**
  Results visualized and compared across multiple runs.

---

### Bonus Exercise: Custom Profiler Tool
**Location:** `A1/B/profiler.py`

#### Features:
- Uses Python's `threading` module to periodically sample CPU usage.
- Saves samples in a `samples` list.
- Supports `plot()` and `summary_table()` methods.
  - **Plot:** Combined graph of core usage over time.
  - **Summary Table:** Core usage averages (minimum excluded due to consistent 0%).

#### Results:
- **JuliaSet Code:**
  - Interval: 0.1 seconds.
  - Usage percentage plotted for all cores.
- **Diffusion Equation Code:**
  - Interval: 0.5 seconds.
  - Visualized core usage trends.

---

ProductVersion:    	15.0.1
BuildVersion:    	24A348
Darwin Mac 24.0.0 Darwin Kernel Version 24.0.0: Tue Sep 24 23:37:13 PDT 2024; root:xnu-11215.1.12~1/RELEASE_ARM64_T8112 arm64

time.time: 7.152557373046875e-07
timeit.default_timer: 8.297502063214779e-08
time.time_ns: 768.0

ProductName:		macOS
ProductVersion:	14.6.1
BuildVersion:		23G93
Darwin Mac 23.6.0 Darwin Kernel Version 23.6.0: Mon Jul 29 21:14:30 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T6030 arm64
time.time: 7.152557373046875e-07
timeit.default_timer: 8.288770914077759e-08
time.time_ns: 768.0

The code below is our clock granularity check used for the results above.


In [ ]:
import numpy as np
import time
import timeit


def checktick(timestampFunction):
    M = 200
    timesfound = np.empty((M,))
    for i in range(M):
        t1 = timestampFunction()  # get timestamp from timer
        t2 = timestampFunction()  # get timestamp from timer
        while (
            t2 - t1
        ) < 1e-16:  # if zero then we are below clock granularity, retake timing
            t2 = timestampFunction()  # get timestamp from timer
        t1 = t2  # this is outside the loop
        timesfound[i] = t1  # record the time stamp
    minDelta = 1000000
    Delta = np.diff(timesfound)  # it should be cast to int only when needed
    minDelta = Delta.min()
    return minDelta


def main():
    print(checktick(time.time))
    print(checktick(timeit.default_timer))
    print(checktick(time.time_ns))


if __name__ == "__main__":
    main()


: 